In [8]:
# Cell 1: Import Libraries and Setup
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Print TensorFlow version and GPU availability
print(f"✅ TensorFlow version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"🚀 GPU is available: {gpus[0].name}")
else:
    print("⚠️ No GPU found. Training will use CPU.")


✅ TensorFlow version: 2.10.1
🚀 GPU is available: /physical_device:GPU:0


In [2]:
# Cell 2: GPU Configuration and Memory Growth
# Configure GPU memory growth to avoid OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Num GPUs Available: {len(gpus)}")
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU available, using CPU")

Num GPUs Available: 1
GPU memory growth enabled


In [3]:
# Cell 3: Configuration Parameters
IMAGE_H = 256
IMAGE_W = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50
LEARNING_RATE = 0.0001

In [4]:
# Cell 4: Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [5]:
# Cell 5: Validation Data Generator
validation_datagen = ImageDataGenerator(
    rescale=1./255
)


In [6]:
# Cell 6: Test Data Generator
test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
# Cell 7: Create Data Generators
train_generator = train_datagen.flow_from_directory(
    'deepfake_final_exam/train',
    target_size=(IMAGE_H, IMAGE_W),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    'deepfake_final_exam/val',
    target_size=(IMAGE_H, IMAGE_W),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    'deepfake_final_exam/test',
    target_size=(IMAGE_H, IMAGE_W),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'deepfake_final_exam/train'

In [ ]:
# Cell 8: Get Class Names
class_names = list(train_generator.class_indices.keys())
n_classes = len(class_names)
print(f"Classes: {class_names}")
print(f"Number of classes: {n_classes}")

In [ ]:
# Cell 9: Display Sample Images
def display_sample_images():
    plt.figure(figsize=(15, 10))
    sample_batch = next(train_generator)
    images, labels = sample_batch
    
    for i in range(min(12, len(images))):
        plt.subplot(3, 4, i + 1)
        plt.imshow(images[i])
        class_idx = np.argmax(labels[i])
        plt.title(f'Class: {class_names[class_idx]}')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

display_sample_images()

In [ ]:
# Cell 10: Build EfficientNetV2 Model with Transfer Learning
def create_efficientnet_model():
    # Load pre-trained EfficientNetV2B0 without top layers
    base_model = EfficientNetV2B0(
        weights='imagenet',
        include_top=False,
        input_shape=(IMAGE_H, IMAGE_W, CHANNELS)
    )
    
    # Freeze base model layers initially
    base_model.trainable = False
    
    # Add custom classification head
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(n_classes, activation='softmax')
    ])
    
    return model

model = create_efficientnet_model()

In [ ]:
# Cell 11: Model Summary
model.summary()


In [ ]:
# Cell 12: Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)


In [ ]:
# Cell 13: Setup Callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        'best_deepfake_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
]

In [ ]:
# Cell 14: Calculate Steps per Epoch
steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = validation_generator.samples // BATCH_SIZE

print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")

In [ ]:
# Cell 15: Initial Training (Feature Extraction)
print("Starting initial training with frozen base model...")
history_1 = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Cell 16: Fine-tuning - Unfreeze Base Model
print("Unfreezing base model for fine-tuning...")
model.layers[0].trainable = True

# Use a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE/10),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

In [ ]:
# Cell 17: Fine-tuning Training
print("Starting fine-tuning...")
history_2 = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Cell 18: Plot Training History
def plot_training_history(history_1, history_2):
    # Combine histories
    acc = history_1.history['accuracy'] + history_2.history['accuracy']
    val_acc = history_1.history['val_accuracy'] + history_2.history['val_accuracy']
    loss = history_1.history['loss'] + history_2.history['loss']
    val_loss = history_1.history['val_loss'] + history_2.history['val_loss']
    
    epochs_range = range(len(acc))
    
    plt.figure(figsize=(16, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.axvline(x=20, color='r', linestyle='--', alpha=0.7, label='Fine-tuning Start')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.axvline(x=20, color='r', linestyle='--', alpha=0.7, label='Fine-tuning Start')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    
    plt.tight_layout()
    plt.show()

plot_training_history(history_1, history_2)


In [ ]:
# Cell 19: Load Best Model and Evaluate
model.load_weights('best_deepfake_model.h5')
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator, verbose=1)

print(f"\nTest Results:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1-Score: {2 * (test_precision * test_recall) / (test_precision + test_recall):.4f}")


In [ ]:
# Cell 20: Prediction Function
def predict_image(model, img_array):
    """Predict single image"""
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array, verbose=0)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * np.max(predictions[0]), 2)
    return predicted_class, confidence

In [ ]:
# Cell 21: Display Test Predictions
def display_predictions():
    plt.figure(figsize=(15, 15))
    
    # Get a batch of test images
    test_images, test_labels = next(test_generator)
    
    for i in range(min(9, len(test_images))):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(test_images[i])
        
        # Get predictions
        predicted_class, confidence = predict_image(model, test_images[i])
        actual_class = class_names[np.argmax(test_labels[i])]
        
        # Color code: green for correct, red for incorrect
        color = 'green' if predicted_class == actual_class else 'red'
        
        plt.title(f"Actual: {actual_class}\nPredicted: {predicted_class}\nConfidence: {confidence}%", 
                 color=color, fontsize=10)
        plt.axis("off")
    
    plt.tight_layout()
    plt.show()

display_predictions()

In [ ]:
# Cell 22: Create Confusion Matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

def create_confusion_matrix():
    # Get all predictions
    y_pred = []
    y_true = []
    
    for images, labels in test_generator:
        predictions = model.predict(images, verbose=0)
        y_pred.extend(np.argmax(predictions, axis=1))
        y_true.extend(np.argmax(labels, axis=1))
    
    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

create_confusion_matrix()

In [ ]:
# Cell 23: Save Model in Different Formats
# Save complete model
model.save('deepfake_efficientnet_model.h5')
print("Model saved as deepfake_efficientnet_model.h5")

# Save as SavedModel format
model.save('deepfake_efficientnet_savedmodel')
print("Model saved as deepfake_efficientnet_savedmodel")

In [ ]:
# Cell 24: Convert to TensorFlow Lite
def convert_to_tflite():
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()
    
    with open("deepfake_efficientnet_model.tflite", 'wb') as f:
        f.write(tflite_model)
    
    print("TensorFlow Lite model saved as deepfake_efficientnet_model.tflite")
    
    # Get model size
    import os
    size_mb = os.path.getsize("deepfake_efficientnet_model.tflite") / (1024 * 1024)
    print(f"TFLite model size: {size_mb:.2f} MB")

convert_to_tflite()


In [ ]:
# Cell 25: Model Performance Summary
print("\n" + "="*50)
print("MODEL PERFORMANCE SUMMARY")
print("="*50)
print(f"Architecture: EfficientNetV2B0 + Custom Head")
print(f"Input Shape: {IMAGE_H}x{IMAGE_W}x{CHANNELS}")
print(f"Classes: {class_names}")
print(f"Total Parameters: {model.count_params():,}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print("="*50)